<a href="https://colab.research.google.com/github/npr99/PlanningMethods/blob/master/PLAN604_0av2_US_Census_Geography.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Understanding US Census Geography

This program focuses on obtaining US Census Geography shapefiles.


In [ ]:
# Install packages not included in Google Colab (-q = option quiet)
!pip -q install geopandas

# Need to upgrade to folium 0.11.0
!pip install --upgrade folium

# Import packages
import sys # For checking version of python for replication
import os   # For saving output to path
import pandas as pd # For reading, writing and wrangling data
import geopandas as gpd # For creating maps
import folium as fm # For creating the final HTML map complete with OSM (Open Street Map) basemap
from folium import plugins # Add minimap and search plugin functions to maps

from google.colab import files  # Required to automatedownload final files 

     |████████████████████████████████| 972kB 2.7MB/s 
     |████████████████████████████████| 10.9MB 15.6MB/s 
     |████████████████████████████████| 14.8MB 238kB/s 
     |████████████████████████████████| 102kB 2.5MB/s 
ERROR: datascience 0.10.6 has requirement folium==0.2.1, but you'll have folium 0.11.0 which is incompatible.
  Found existing installation: folium 0.8.3
    Uninstalling folium-0.8.3:
      Successfully uninstalled folium-0.8.3


In [ ]:
print("Python Version     ", sys.version)
print("pandas version:    ", pd.__version__)
print("geopandas version: ", gpd.__version__)
print("folium version:    ", fm.__version__)

Python Version      3.6.9 (default, Jul 17 2020, 12:50:27) 
[GCC 8.4.0]
pandas version:     1.0.5
geopandas version:  0.8.1
folium version:     0.11.0


Using the wget command download the cartographic boundaries files from the US Census website.

The cartographic files come in three levels of resolution. The "smallest" file size will be the lowest resolution - 20m = 1:20,000,000. For smaller geographies the 5m = 1:5,000,000 resolution and the 500k = 1:500,000 resolutions provide more details.

In [ ]:
# Program Prepare Geography combines steps needed to prepare files for mapping
def prepare_geography(year,state,entity,resolution):
  filename = f'cb_{year}_{state}_{entity}_{resolution}'

  !wget -q https://www2.census.gov/geo/tiger/GENZ{year}/shp/{filename}.zip

  !unzip -o -q {filename}.zip
  gdf = gpd.read_file(f'{filename}.shp')
  gdf = gdf.to_crs(epsg=4326)
  
  return gdf

## Create a list of the geographies to be prepared
The list includes the geographic entity and the resolution.

In [ ]:
state = '48'
entities = [[state,'tract','500k'],
            [state,'place','500k'],
            [state,'county_within_ua','500k']]

In [ ]:
geographies = {}
for entity in entities:
    geographies[entity[1]] = prepare_geography('2019',entity[0],entity[1],entity[2])

    # Approximate measure of sq km area based on Equal Area Cylindrical projection (cea)
    # https://proj.org/operations/projections/cea.html
    geographies[entity[1]]= geographies[entity[1]].to_crs({'proj':'cea'})
    geographies[entity[1]]["areasqkm"] = geographies[entity[1]]['geometry'].area/ 10**6

    # Set geography to EPSG 4326
    # EPSG: 4326 uses a coordinate system  (Lat, Lon) on the surface of a sphere
    # with the WGS84 datum. Open Street Map and Google Earth use 4326.
    geographies[entity[1]]= geographies[entity[1]].to_crs(epsg = 4326)

    count = len(geographies[entity[1]])
    print('US Census Geographic Entity:',entity[1], '\t count = ',count,'\t\t state = ',entity[0])

US Census Geographic Entity: tract 	 count =  5254 		 state =  48
US Census Geographic Entity: place 	 count =  1746 		 state =  48
US Census Geographic Entity: county_within_ua 	 count =  375 		 state =  48


In [ ]:
def create_column(entity,reference_entity):
  column = geographies[entity][["areasqkm"]].describe()
  column = column.rename(columns = {"areasqkm" : entity})
  column.loc["Total"] = geographies[entity]["areasqkm"].sum()

  column.loc["% Total"] = geographies[entity]["areasqkm"].sum() / geographies[reference_entity]["areasqkm"].sum() * 100

  return column

columns = {}
for entity in entities:
  columns[entity[1]] = create_column(entity[1],'tract')

In [ ]:
table1 = pd.concat([columns['tract'], 
                    columns['place'], 
                    columns['county_within_ua']], axis=1)
varformat = "{:,.0f}"
table_title = "Table 1. Descriptive statistics for approximate area (sq km) for US Census Geographies."
table1 = table1.style.set_caption(table_title).format(varformat).set_properties(**{'text-align': 'right'})
table1

,tract,place,county_within_ua
count,"5,254","1,746",375
mean,131,20,61
std,547,75,238
min,0,0,0
25%,2,3,5
50%,5,6,10
75%,28,14,26
max,"16,016","1,740","3,217"
Total,"687,749","35,734","22,975"
% Total,100,5,3


In [ ]:
# Create a guide for the map styles for each entity
# Colors from Color Brewer
# https://colorbrewer2.org/#type=sequential&scheme=BuGn&n=5

maplayers = [['place','tract']]

# Make directory to save output
if not os.path.exists('output_maplayers'):
    os.mkdir('output_maplayers')

In [ ]:
style_function1 = lambda x: {
            'fillColor': 'transparent',
            'color': 'red',
            'weight': 4,
            'fillOpacity': 0
        }
style_function2 = lambda x: {
            'fillColor': 'green',
            'color': 'black',
            'weight': 1,
            'fillOpacity': 0.25
        }

highlight_function1 = lambda x: {'fillOpacity': 0.5}

maps = {}
for layer in maplayers:
    entity1 = layer[0]
    entity2 = layer[1]
    mapname = entity1+entity2
    print(mapname)

    # What location should the map be centered on?
    center_x = (geographies[entity2].bounds.minx.mean() + geographies[entity2].bounds.maxx.mean())/2
    center_y = (geographies[entity2].bounds.miny.mean() + geographies[entity2].bounds.maxy.mean())/2
    print(f'The center of the {entity2} data file is located at {center_x} {center_y}')

    maps[mapname] = fm.Map(location=[center_y, center_x], 
        max_bounds = True,
        control_scale=True,
        prefer_canvas=True)

    baselayer = fm.GeoJson(
            geographies[entity1].to_json(),
            name= entity1,
            style_function= style_function1,
            tooltip=fm.features.GeoJsonTooltip(fields=['NAME','AFFGEOID','GEOID'], 
                                               sticky=True,
                                               )                
        ).add_to(maps[mapname])

    searchlayer = fm.GeoJson( 
            geographies[entity2].to_json(),
            name= entity2,
            style_function=style_function2,
            highlight_function=highlight_function1,
            tooltip=fm.features.GeoJsonTooltip(fields=['AFFGEOID','GEOID'], sticky=False) ,
            popup= fm.GeoJsonPopup(
                fields=['GEOID']
            )
        ).add_to(maps[mapname] )

    fm.LayerControl(collapsed=False, autoZIndex=False).add_to(maps[mapname])

    # Add minimap
    plugins.MiniMap().add_to(maps[mapname])

    # Add search function
    plugins.Search(searchlayer,position='topleft',
                           search_zoom=15,
                           search_label='GEOID',
                           geom_type='Polygon').add_to(maps[mapname])

    # How should the map be bound - look for the southwest and northeast corners of the data
    sw_corner = [geographies[entity2].bounds.miny.min(),geographies[entity2].bounds.minx.min()]
    ne_corner = [geographies[entity2].bounds.maxy.max(),geographies[entity2].bounds.maxx.max()]
    print(f'The {entity2} data file is bounded by at {sw_corner} {ne_corner}')
    maps[mapname].fit_bounds([sw_corner, ne_corner])

    maps[mapname].save(f'output_maplayers/{mapname}.html')
    #files.download(f'output_maplayers/{mapname}.html')

placetract
The center of the tract data file is located at -97.4714547089282 31.010524233089118
The tract data file is bounded by at [25.83737698658744, -106.645646] [36.50070399140901, -93.508292]


In [ ]:
display(maps['placetract'])

In [ ]:
geographies['place'].head()

,STATEFP,PLACEFP,PLACENS,AFFGEOID,GEOID,NAME,LSAD,ALAND,AWATER,geometry,areasqkm
0,48,41464,02411626,1600000US4841464,4841464,Laredo,25,273872753,3805095,"POLYGON ((-99.74828 27.74440, -99.74624 27.742...",277.655947
1,48,45744,02411064,1600000US4845744,4845744,McKinney,25,173372597,1916548,"MULTIPOLYGON (((-96.58829 33.22624, -96.58270 ...",175.541192
2,48,48072,02411096,1600000US4848072,4848072,Midland,25,192650681,441855,"POLYGON ((-102.26976 31.97002, -102.25282 31.9...",193.072310
3,48,53388,02411303,1600000US4853388,4853388,Odessa,25,117128032,667150,"POLYGON ((-102.44136 31.83506, -102.43407 31.8...",117.842137
4,48,61196,02411530,1600000US4861196,4861196,Red Oak,25,40280234,13667,"MULTIPOLYGON (((-96.72648 32.47851, -96.72432 ...",40.296520
